# 2차 프로그램 튜터리얼(Quadratic Program (QP) Tutorial)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

## Important Note
Drake에서 일반적인 최적화 프로그램 생성 및 풀기 위해서 [mathematical program tutorial](./mathematical_program.ipynb)를 참고하자.

볼록(convex) 2차 프로그램 (QP)은 볼록 최적화(convex optimization)의 특수한 유형이다. 비용 함수는 볼록 2차 함수이며, 제약 조건은 선형, 즉 선형 계획법의 제약 조건과 동일하다. 볼록 2차 계획법은 다음과 같은 형태를 갖는다.

$\begin{aligned}
\min_x \quad & 0.5 x^TQx + b^Tx + c\\
\text{s.t.} \quad & Ex \leq f
\end{aligned}$

여기서 `Q`는 양의 준정부호 행렬(position semidefinite matrix)이다.

Drake는 OSQP, SCS, Gurobi, MOSEK™ 등 다양한 솔버를 통해 2차 프로그램 문제를 해결할 수 있다. 지원되는 솔버 전체 목록은 [Doxygen page]( https://drake.mit.edu/doxygen_cxx/group__solvers.html)를 참조하자. 일부 상용 솔버 (Gurobi, MOSEK™ 등)는 사전 컴파일된 Drake 바이너리에 포함되지 않아 Deepnote/Colab/Binder에서 사용할 수 없다.

Drake API는 2차 비용(quadratic cost)과 선형 제약 조건을 추가하는 다양한 함수를 지원한다. 이 튜터리얼에서는 몇 가지 기능을 간략하게 살펴보자. 전체 기능 목록은 [Doxygen](https://drake.mit.edu/doxygen_cxx/classdrake_1_1solvers_1_1_mathematical_program.html)을 확인하자.

QP는 로봇 공학에서 다양한 응용 분야를 가지고 있다. 예를 들어, 차분 역 운동학 문제(differential inverse kinematics problem)를 QP로 해결할 수 있다. 자세한 내용은 [DifferentialInverseKinematics](https://drake.mit.edu/doxygen_cxx/group__planning__kinematics.html#gac36361db531cfc7f707ef52db81375e9)를 참조하자. 더 많은 예시는 [Underactuated Robotics code repo](https://github.com/RussTedrake/underactuated)를 참고하자.

## 2차 비용 추가하기(Add quadratic cost)

### AddQuadraticCost function

2차 비용(quadratic cost)을 추가하는 가장 쉬운 방법은 `AddQuadraticCost` 함수를 호출하는 것이다. 다음 코드에서는 먼저 2개의 의사 결정 변수가 있는 프로그램을 구성한 다음 `AddQuadraticCost` 함수를 호출하는 방법을 보여준다.

In [ ]:
from pydrake.solvers import MathematicalProgram, Solve
import numpy as np

# Create an empty MathematicalProgram named prog (with no decision variables,
# constraints or costs)
prog = MathematicalProgram()
# Add two decision variables x[0], x[1].
x = prog.NewContinuousVariables(2, "x")

`AddQuadraticCost(expression)` 함수를 호출하여 2차 비용을 추가할 수 있다. 여기서 `expression`은 2차 비용을 나타내는 기호식 표현이다.

In [ ]:
# Add a symbolic quadratic expression as the quadratic cost.
cost1 = prog.AddQuadraticCost(x[0]**2 + 2*x[0]*x[1] + x[1]**2 + 3*x[0] + 4)
# The newly added cost is returned as cost1
print(cost1)
# The newly added cost is stored inside prog.
print(prog.quadratic_costs()[0])

다시 `AddQuadraticCost`를 호출할 경우, `prog`의 총 비용은 추가된 모든 비용의 합. `prog.quadratic_costs()`가 2개의 항목을 가진 것을 확인할 수 있다. 그리고 `prog`의 총 비용은 `cost1 + cost2`이다.

In [ ]:
# Add another quadratic cost to prog.
cost2 = prog.AddQuadraticCost(x[1]*x[1] + 3)
print(f"The number of quadratic costs in prog: {len(prog.quadratic_costs())}")

2차 비용의 계수 `Q, b, c`를 알고 있다면 기호식을 사용하지 않고도 비용을 추가할 수 있다. 다음 코드와 같이 

In [ ]:
# Add the cost x[0]*x[0] + x[0]*x[1] + 1.5*x[1]*x[1] + 2*x[0] + 4*x[1] + 1
cost3 = prog.AddQuadraticCost(Q=[[2, 1], [1, 3]], b=[2, 4], c=1, vars=x)
print(f"cost 3 is {cost3}")

### AddQuadraticErrorCost
2차 비용을 추가할 수 있다.

$\begin{aligned}
(x - x_{desired})^TQ(x-x_{desired})
\end{aligned}$

`AddQuadraticErrorCost`를 호출해서 프로그램에 추가 가능. 예제 코드는

In [ ]:
# Adds the cost (x - [1;2])' * Q * (x-[1;2])
cost4 = prog.AddQuadraticErrorCost(Q=[[1, 2],[2, 6]], x_desired=[1,2], vars=x)
print(f"cost4 is {cost4}")

### Add2NormSquaredCost
2차 비용을 추가할 수 있다.

$\begin{aligned}
|Ax-b|^2
\end{aligned}$

벡터 `Ax-b`의 제곱된 L2 norm(2차 norm)을 `Add2NormSquaredCost` 함수를 호출하여 프로그램에 추가할 수 있다. 다음은 코드 예시입니다.

In [ ]:
# Adds the squared norm of (x[0]+2*x[1]-2, x[1] - 3) to the program cost.
cost5 = prog.Add2NormSquaredCost(A=[[1, 2], [0, 1]], b=[2, 3], vars=x)
print(f"cost5 is {cost5}")

## Add linear cost

2차 프로그램 문제에 선형 비용을 추가할 수도 있다. 선형 비용 추가 시 사용되는 다양한 API에 대한 소개는 [linear programming tutorial](./linear_program.ipynb)을 참조하십시오. 다음과 같은 예제가 있다.

In [ ]:
# Adds a linear cost to the quadratic program
cost6 = prog.AddLinearCost(x[0] + 3 * x[1] + 1)
print(f"cost6 is {cost6}")
print(f"Number of linear costs in prog: {len(prog.linear_costs())}")

## Add linear constraints

2차 프로그램 문제에 선형 제약 조건을 추가하려면 [linear programming tutorial](./linear_program.ipynb)의 `선형 제약 조건 추가(Add linear constraints)` 섹션을 참조하세요. 다음은 간단한 예시입니다.

In [ ]:
constraint1 = prog.AddLinearConstraint(x[0] + 3*x[1] <= 5)
# Adds the constraint 1 <= x[0] <= 5 and 1 <= x[1] <= 5
constraint2 = prog.AddBoundingBoxConstraint(1, 5, x)

## Convexity of the cost
Drake는 각 2차 비용의 헤시안(Hessian) 행렬이 양의 semidefinite 인지 아닌지 검사하여 모든 추가된 2차 비용의 볼록성(convexity)을 확인한다. 현재 사용자는 모든 2차 비용이 볼록한 경우에만 볼록 솔버(Gurobi/MOSEK™/OSQP/CLP/SCS 등)를 사용하여 QP를 해결할 수 있다.

2차 비용이 볼록한지 확인하려면 계산이 필요하다. 애플리케이션에서 가능한 한 빠르게 QP를 해결해야 하는 경우(예: 로봇 제어 루프 내에서 QP 해결) 볼록성 검사를 건너뛰는 것이 합리적일 수 있다. 이를 위해 2차 비용의 볼록성을 알고 있다면 `AddQuadraticCost` 함수에서 `is_convex` 플래그를 설정할 수 있다. `is_convex` 플래그가 설정되면 Drake는 비용이 볼록한지 여부를 계산하지 않는다. 다음 예시에서는 이 `is_convex` 플래그를 설정하는 방법을 보여준다.

In [ ]:
# Call AddQuadraticCost with the is_convex flag unspecified. Drake will check the convexity
# of this cost.
cost7 = prog.AddQuadraticCost(x[0]**2 + 3 * x[1]**2 + x[0] * x[1] + 2 * x[0])
print(f"Is the cost {cost7} convex? {cost7.evaluator().is_convex()}")
cost8 = prog.AddQuadraticCost(x[0]**2 + 3 * x[1]**2 + 4*x[0]*x[1])
print(f"Is the cost {cost8} convex? {cost8.evaluator().is_convex()}")

# Call AddQuadraticCost with specified is_convex flag. Drake won't check the convexity of
# the quadratic cost when this flag is specified.
cost9 = prog.AddQuadraticCost(x[0]**2 + 4 * x[1]**2 + 3 * x[0]*x[1], is_convex=True)
print(f"Is the cost {cost9} convex? {cost9.evaluator().is_convex()}")

## 코드 예제 완성(A complete code example)
아래에서 QP를 생성하고 해결하는 완성된 예시를 보자.

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(3, "x")
prog.AddQuadraticCost(x[0] * x[0] + 2 * x[0] + 3)
# Adds the quadratic cost on the squared norm of the vector
# (x[1] + 3*x[2] - 1, 2*x[1] + 4*x[2] -4)
prog.Add2NormSquaredCost(A = [[1, 3], [2, 4]], b=[1, 4], vars=[x[1], x[2]])

# Adds the linear constraints.
prog.AddLinearEqualityConstraint(x[0] + 2*x[1] == 5)
prog.AddLinearConstraint(x[0] + 4 *x[1] <= 10)
# Sets the bounds for each variable to be within [-1, 10]
prog.AddBoundingBoxConstraint(-1, 10, x)

# Solve the program.
result = Solve(prog)
print(f"optimal solution x: {result.GetSolution(x)}")
print(f"optimal cost: {result.get_optimal_cost()}")

QP에 관현 더 상세한 내용은 아래를 참고하자.

[Quadratic Programming wiki](https://en.wikipedia.org/wiki/Quadratic_programming)

[Numerical Optimization by J. Nocedal and S.Wright](http://www.apmath.spbu.ru/cnsa/pdf/monograf/Numerical_Optimization2006.pdf)